Creating Reviewer-Recipe-Rating matrix :

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
final_processed_dataset=pd.read_csv(r"Final_Processed_Dataset.csv")

reviewer_recipe_rating_matrix = final_processed_dataset.pivot_table(index='ReviewerName', columns='Name', values='Rating')
reviewer_recipe_rating_relation_matrix=reviewer_recipe_rating_matrix.fillna(0)

Calculating cosine similarity matrix :

In [3]:
cosine_similarity_matrix=cosine_similarity(reviewer_recipe_rating_relation_matrix)

cosine_similarity_matrix = pd.DataFrame(cosine_similarity_matrix, index=reviewer_recipe_rating_relation_matrix.index, columns=reviewer_recipe_rating_relation_matrix.index)

Predicting Ratings for all recipes where rating is nan for a each user one by one :

In [4]:
reviewer_recipe_rating_matrix

Name,"""A Bit Different"" Breakfast Pizza","""Better Then Bush's"" Baked Beans","""Chicken","""Chinese"" Candy","""Chinese"" chop suey","""Crock Pot Corned Beef","""Easiest Ever"" Hollandaise Sauce","""How I got my family to eat spinach"" Spinach Casserole","""I Don't Feel Like Cooking Tonight"" Casserole","""Keep it Going"" German Friendship Cake",...,sweet pasta salad,texas brownies,thai seafood stew,tiramisu for atkins dieters,"tomato, cucumber, & onion salad",traditional Irish Ham and Cabbage Dinner,traditional spaghetti bolognese,turkey croquettes,twice baked bacon and cheese potatoes,watermelon chiller
ReviewerName,,,,,,,,,,,,,,,,,,,,,
.. Ameera ..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pamela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Poppy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
- Carla -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
znglass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zoe.s.true,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zombiesgirl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
